In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import time

# hyperparameters
eval_iters = 200
batch_size = 16
block_size = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embd = 128
n_layers = 4
# learning_rate = 0.2
learning_rate = 1e-3
n_heads = 16
dropout = 0.4
eval_interval = 100
max_iters = 100

# # hyperparameters
# batch_size = 16
# block_size = 32
# max_iters = 100
# eval_interval = 100
# learning_rate = 1e-3
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 200
# n_embd = 64
# n_head = 16
# n_layer = 8
# dropout = 0.2
# #-----------


# torch.manual_seed(1337)

# get data
text = open('../data/shakespeare.txt', 'r', encoding='utf-8').read()


# set of all possible characters
chars = sorted(list(set(text)))
vocab_size = len(chars)


# transformations encoding and decoding
itos = {i: s for i, s in enumerate(chars)}
stoi = {s: i for i, s in enumerate(chars)}
encode = lambda s: list(stoi[c] for c in s)
decode = lambda l: ''.join(itos[i] for i in l)


#create data splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]



# get a batch of data
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
    


# estimate the loss
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


# create Head of attention
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        
        # compute attention scores
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        # compute the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out
    
# define multihead attention
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
    
    

# define feed forward class
class FeedForward(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        x = self.net(x)
        return x


# define block
class Block(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        head_size = n_embd // n_heads
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        
        out = x + self.sa(self.ln1(x))
        out = out + self.ffwd(self.ln2(out))
        
        return out
    
# define language model class
class NGramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, x, targets=None):
        
        B,T = x.shape
        
        tok_emb = self.token_embedding_table(x)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        
        if targets is None:
            loss = None
        else:
        
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
    
        for _ in range(max_new_tokens):
            
            # crop to block size
            idx_cond = idx[:, -block_size:]
            # make prediction
            logits, loss = self(idx_cond)
            # select last elemnt in sequence
            logits = logits[:, -1, :]
            #get probs
            probs = F.softmax(logits, dim=-1)
            # sample token
            idx_next = torch.multinomial(probs, 1)
            # append to end of sequence
            idx = torch.cat((idx, idx_next), dim=1)
        
        return idx
    

model = NGramLanguageModel()
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
            
print(sum(p.numel() for p in model.parameters()) / 1e6, ' million parameters')

0.816705  million parameters


In [3]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, 10)[0].tolist()))


Cnqwd$:Tyl


In [8]:
max_iters = 2000
start_time = time.time()
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
stop_time = time.time()
print(f'total time = {stop_time-start_time} seconds')

step 0: train loss 1.7019, val loss 1.8690
step 100: train loss 1.6946, val loss 1.8405
step 200: train loss 1.6921, val loss 1.8633
step 300: train loss 1.6880, val loss 1.8504
step 400: train loss 1.6852, val loss 1.8571
step 500: train loss 1.6826, val loss 1.8471
step 600: train loss 1.6755, val loss 1.8431
step 700: train loss 1.6729, val loss 1.8463
step 800: train loss 1.6667, val loss 1.8457
step 900: train loss 1.6689, val loss 1.8319
step 1000: train loss 1.6633, val loss 1.8261
step 1100: train loss 1.6593, val loss 1.8391
step 1200: train loss 1.6611, val loss 1.8330
step 1300: train loss 1.6434, val loss 1.8240
step 1400: train loss 1.6423, val loss 1.8178
step 1500: train loss 1.6536, val loss 1.8250
step 1600: train loss 1.6412, val loss 1.8173
step 1700: train loss 1.6422, val loss 1.8069
step 1800: train loss 1.6347, val loss 1.8169
step 1900: train loss 1.6307, val loss 1.8091
step 1999: train loss 1.6334, val loss 1.8139
total time = 246.05122637748718 seconds


In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, 500)[0].tolist()))



LUCIOIO:
I murt you be done uppather his and hears acce:
Coliturer one the good cown:
Yet not facher extileo, on wrike, miness herds
To truch for enchance, then O sword to a word divece
Work, too what for son'd the haws swen leggt me stuns.

DUKE OF YORK:
Selsidom my recte will grace of her, as at areate
Isarethan unhambod rustacil. O, not mine?

My PORIOLANUL:

My OF CORIO:
Lhomge, ale hus, it not be the pityber and your joWing my ball:
How shome book, Comfor his to subalen
Father thinke fruit
